<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       DurbinWatson Function in Vantage
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial'><b>Introduction</b></p>
<p style = 'font-size:18px;font-family:Arial'><b>DurbinWatson</b></p>
<p style = 'font-size:16px;font-family:Arial'>The DurbinWatson() function determines if there is evidence of serial correlation between residuals in a residual series. Serial correlation occurs when the residuals of a model are dependent and go against an assumption of linear regression.</p>

<p style = 'font-size:16px;font-family:Arial'>This function accepts inputs from tables referenced by a TDSeries, and TDAnalyticResult that includes an fitresidual layer. It takes values between 0 and 4, with a value of 2 indicating no serial correlation. Values between 0 and 2 indicate positive serial correlation. Values between 2 and 4 indicate negative serial correlation.</p>

<p style = 'font-size:16px;font-family:Arial'>The test statistic is based on the sum of squared differences between adjacent residuals in a model. The statistic is compared to critical values that depend on the sample size and the number of independent variables in the model. If the test statistic is significantly different from 2, then it indicates that the residuals are serial correlated, and the regression model may not be reliable. In this case, you need to change the model or employ additional techniques to account for the serial correlation..</p>

<p style = 'font-size:16px;font-family:Arial'>The following procedure is an example of how to use DurbinWatson() function:</p>
<li style = 'font-size:16px;font-family:Arial'>Use LinearRegr() on the residuals.</li>
<li style = 'font-size:16px;font-family:Arial'>Use DurbinWatson() on the output from LinearRegr() to determine</li>         


<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>1. Initiate a connection to Vantage</b>

<p style = 'font-size:16px;font-family:Arial'>In the section, we import the required libraries and set environment variables and environment paths (if required).

In [ ]:
from teradataml import (
    create_context,
    execute_sql,
    load_example_data,
    DataFrame,
    in_schema,
    TDSeries,
    LinearRegr,
    db_drop_table,
    DurbinWatson,
    db_drop_view,
    remove_context
    )
from teradatasqlalchemy.types import *
# Modify the following to match the specific client environment settings
display.max_rows = 5

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>1.1 Connect to Vantage</b></p>
<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../../UseCases/startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=PP_DurbinWatson.ipynb;' UPDATE FOR SESSION; ''')

<p style = 'font-size:16px;font-family:Arial'>Begin running steps with Shift + Enter keys. </p>

<hr style='height:1px;border:none;background-color:#00233C;'>

<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>1.2 Getting Data for This Demo</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here, we will get the time series data which is available in the teradataml library and use the same to show the usage of the function.</p>

In [ ]:
load_example_data("uaf", ["house_values2"])

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>2. Data Exploration</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Create a "Virtual DataFrame" that points to the data set in Vantage. Check the shape of the dataframe as check the datatype of all the columns of the dataframe.</p>

In [ ]:
data = DataFrame.from_table("house_values2")
data

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>3. DurbinWatson</b><p></p>
<p style = 'font-size:16px;font-family:Arial'>The DurbinWatson() function determines serial correlation between residuals within an independent time series data, or in the residual result of TDAnalyticResult.</p>

In [ ]:
linearregr_series = TDSeries(data=data,
                                 id="cid",
                                 row_index_style="SEQUENCE",
                                 row_index="s_no",
                                 payload_field=["house_value", "salary"],
                                 payload_content="MULTIVAR_REAL"
                                 )

<p style = 'font-size:16px;font-family:Arial'>The LinearRegr() function fits TDSeries data to the curve mentioned in the "formula. It returns a result containing solved coefficients, model statistics, and residuals statistics.</p>

In [ ]:
linearregr_output = LinearRegr(data=linearregr_series,
                                   variables_count=2,
                                   weights=False,
                                   formula="Y=B0+B1*X1",
                                   algorithm='QR',
                                   model_stats=True,
                                   coeff_stats=False,
                                   residuals=True
                                   )

<p></p>
<p style = 'font-size:16px;font-family:Arial'>Detailed help can be found by passing function name to built-in help function. </p>

In [ ]:
help(DurbinWatson)

<p style = 'font-size:16px;font-family:Arial'>We need to first convert the data from dataframe into a TDSeries which will be passed to the DurbinWatson function as input.</p>

In [ ]:
seriesspec = TDSeries(data=linearregr_output.fitresiduals,
                          id="cid",
                          row_index_style="SEQUENCE",
                          row_index="ROW_I",
                          payload_field="RESIDUAL",
                          payload_content="REAL"
                          )

In [ ]:
uaf_out2 = DurbinWatson(data=seriesspec,
                            explanatory_count=2,
                            method="DW_FORMULA",
                            significance_level=0.05)
# Print the result DataFrame.
uaf_out2.result

<p></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>If the test statistic is significantly different from 2, then it indicates that the residuals are serial correlated, and the regression model may not be reliable. In this case, you need to change the model or employ additional techniques to account for the serial correlation.</p>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>ACCEPT means the null hypothesis is accepted. No Unit roots are present, and therefore the process is stationary.</li>
<li style = 'font-size:16px;font-family:Arial;color:##00233C'>REJECT means the null hypothesis is rejected. Unit roots are present, and the process may or may not be stationary, depending on other factors.</li>
</p>

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>4. Cleanup</b>

<p style = 'font-size:18px;font-family:Arial'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up intermediate tables.</p>

In [ ]:
db_drop_table("house_values2")

In [ ]:
remove_context()

<hr style="height:1px;border:none;">

<p style = 'font-size:16px;font-family:Arial'><b>Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Teradataml Python reference: <a href = 'https://docs.teradata.com/search/all?query=Python+Package+User+Guide&content-lang=en-US'>here</a></li>
    <li>DurbinWatson function reference: <a href = 'https://docs.teradata.com/search/all?query=DurbinWatson&content-lang=en-US'>here</a></li>
</ul>

<footer style="padding-bottom:35px; border-bottom:3px solid #91A0Ab">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved
        </div>
    </div>
</footer>